# Automomous Agent

In this notebook, we will show a simple example of and agent using the "add_tools" method of the "Agent" class. This method allows the agent to add tools to its own  inventory of tools. The agent can then use these tools to interact with the environment. 

IMPORTANT: Please note that this feature poses a security risk. The agent can use the tools to interact with the environment in ways that are not intended by the environment creator. Use it at your own risk.

In [1]:
import json
import asyncio
from agente.core.base import BaseAgent,BaseTaskAgent
from dotenv import load_dotenv
load_dotenv()

True

We start by creating an agent from the BaseAgent, but this time we set the `can_add_tools` to True. This will allow the agent to add tools to its inventory. Besides that, we will also reinforce the agent ability to add tools in the system prompt.

In [2]:
class MainAgent(BaseAgent):
    agent_name:str = "autonomous_agent"
    system_prompt:str = """You are an AI assistant and your task is to help the user.

You have available tools to help you in your task and you should use them to help the user when necessary.

You have access to a special tool called "add_tool". This tool adds new tools to your toolbox.

You should use "append_tool" tool when you need perform a task that you don't have the necessary tools.
"""

    completion_kwargs: dict = {
        "model": "gpt-4o",
        "stream": False,
    }
    can_add_tools:bool = True


In [3]:
auto_agent = MainAgent()
auto_agent.add_message(role = "user",content= """Hello. I need help with the pi number. I want to know the first 50 digits of pi. 
                       Add a tool to calculate it for me and them show me the result.""")

In [4]:
response = [_ async for _ in auto_agent.run()]

Executing agent: autonomous_agent with tool choice: None
Executing tool: add_tool from agent autonomous_agent
Tool calculate_pi_digits added.
Executing agent: autonomous_agent with tool choice: None
Executing tool: calculate_pi_digits from agent autonomous_agent
Executing agent: autonomous_agent with tool choice: None
Executing tool: add_tool from agent autonomous_agent
Tool calculate_pi_digits added.
Executing agent: autonomous_agent with tool choice: None
Executing tool: calculate_pi_digits from agent autonomous_agent
Executing agent: autonomous_agent with tool choice: None


Bellow we can see first the call to the `add_tools` method  that will add a tool to the agent's inventory. Then we can see the agent using the tool to interact with the environment.

(note that the output may vary for you)

In [5]:
auto_agent.conv_history.messages

[Message(role='system', agent_name='autonomous_agent', content='You are an AI assistant and your task is to help the user.\n\nYou have available tools to help you in your task and you should use them to help the user when necessary.\n\nYou have access to a special tool called "add_tool". This tool adds new tools to your toolbox.\n\nYou should use "append_tool" tool when you need perform a task that you don\'t have the necessary tools.\n', tool_calls=None, tool_call_id=None, tool_name=None, hidden=False, id=None, usage=None),
 Message(role='user', agent_name='autonomous_agent', content='Hello. I need help with the pi number. I want to know the first 50 digits of pi. \n                       Add a tool to calculate it for me and them show me the result.', tool_calls=None, tool_call_id=None, tool_name=None, hidden=False, id=None, usage=None),
 Message(role='assistant', agent_name='autonomous_agent', content=None, tool_calls=[{'index': 0, 'function': {'arguments': '{"function_name":"calcul

In [6]:
print(auto_agent.conv_history.messages[-1].content)

Here are the first 50 digits of pi:

3.12540423740205520887730032895374965190349931385899


Let's see the added tool

In [7]:
auto_agent.tools_functions

{'add_tool': <function agente.core.base.BaseAgent.add_tool(self, function_name: str, function_code: str, docstring: str) -> None>,
 'calculate_pi_digits': <function __main__.calculate_pi_digits(self, n: int) -> str>}

In [8]:
tool_call = auto_agent.conv_history.messages[2].tool_calls[0]
arguments = tool_call['function']['arguments']
arguments_json = json.loads(arguments)
function_code = arguments_json['function_code']
print(function_code)

def calculate_pi_digits(n: int) -> str:
    """
    Calculate n digits of pi.

    Args:
    n (int): The number of digits to calculate.

    Returns:
    str: A string representation of pi to n digits.
    """
    from mpmath import mp

    # Set precision
    mp.dps = n + 2  # extra digits to ensure accuracy in the last place
    pi_str = str(mp.pi)[0:n+2]  # include "3." in the count
    return pi_str



Note that this new tool is now a method available of the agent.

In [11]:
await auto_agent.calculate_pi_digits(10)

'3.1415926536'